# IMPORTING THE LIBRARIES

In [37]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import time
import difflib
import numpy
import tflearn
import tensorflow
import json
import pickle
import random
import os

# PRE- PROCESSING THE DATA

In [38]:
with open("intents (1).json") as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels =[]
    docs_patt = []
    docs_tag = []

#TOKENISATION & STEMMING
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            for item in wrds:
                words.extend(wrds)
                docs_patt.append(wrds)
                docs_tag.append(intent["tag"])
                if intent["tag"] not in labels:
                    labels.append(intent["tag"])
    words = [stemmer.stem(w.lower()) for w in words]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]
# BAG OF WORDS - FEATURE ENGINEERING
    for x, doc in enumerate(docs_patt):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]
        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
        output_row = out_empty[:]
        output_row[labels.index(docs_tag[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

# MODEL BUILDING

In [39]:
from tensorflow.python.framework import ops
ops.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

BATCH GRADIENT DESCEND -> BATCH SIZE=8, NO OF EPOCHES= 1000

In [40]:
try:
    model.load("model.tflearn")
except:
    model = tflearn.DNN(net)
    history = model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

Training Step: 217999  | time: 1.363s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9985 -- iter: 1736/1744
Training Step: 218000  | time: 1.366s
| Adam | epoch: 1000 | loss: 0.00000 - acc: 0.9986 -- iter: 1744/1744
--
INFO:tensorflow:C:\Users\Dell\Desktop\major project\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


# INPUT PRE PROCESSING

In [41]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    return numpy.array(bag)

In [42]:
def words_to_list(s):
    a = []
    ns = ""
    s = s + " " 
    for i in range(len(s)):
        if s[i] == " ":
            a.append(ns)
            ns = ""
        else:
            ns = ns + s[i]
    a = list(set(a))
    return a

In [43]:
# pass the file in this fuction to create a dictionary of unique vocabulary
def json_to_dictionary(data):
    dictionary = []
    fil_dict= []
    vocalubary = []
    for i in data["intents"]:
        for pattern in i["patterns"]:
            vocalubary.append(pattern.lower())
    for i in vocalubary:
        dictionary.append(words_to_list(i))
    for i in range(len(dictionary)):
        for word in dictionary[i]:
            fil_dict.append(word)
    return list(set(fil_dict))

# this fuction checks the spelling in the sentence
chatbot_vocabulary = json_to_dictionary(data)

In [44]:
def word_checker(s):
    correct_string = ""
    for word in s.casefold().split():
        if word not in chatbot_vocabulary:
            suggestion = difflib.get_close_matches(word, chatbot_vocabulary)
            for x in suggestion:
                pass
            if len(suggestion) == 0:
                pass
            else:
                correct_string = correct_string + " " + str(suggestion[0])
        else:
            correct_string = correct_string + " " + str(word)

    return correct_string 

# SPEECH RECOGNITION

In [45]:
import speech_recognition as sr
r=sr.Recognizer()
import pyttsx3
engine = pyttsx3.init()

In [46]:
def bot_speaking(message):
    engine.say(message)
    engine.runAndWait()
    if engine._inLoop:
        engine.endLoop()

In [47]:
def get_input():
    with sr.Microphone() as source:
        #print("Say something!!!");
        bot_speaking("Hey mate say something")
        audio=r.listen(source,timeout=0)
        #print("Perfect, Thanks!")
        bot_speaking("Perfect, Thanks!")
    try:
        msg=r.recognize_google(audio)
        print("TEXT: "+msg); #r.recognize(audio,language='hi-IN')
        bot_speaking("you said "+msg)
        return msg
    except:
        #print("Dude it's not working :(")
        bot_speaking("Sorry mate! It's not working")
        pass;

# CHAT FUNCTION

In [48]:
def chat():
    print("VNR :Hi! I am your personal bot. I am here to answer queries on VNRVJIET")
    while True:
        #inp=str(get_input())
        inp=input("YOU : ")
        if inp.lower() == "quit"or inp==None:
            break
        inp_x = word_checker(inp)
        results = model.predict([bag_of_words(inp_x, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(results[results_index])
        if results[results_index] >= 0.9:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms= random.choice(responses)
                    print("VNR : "+ms)
                    bot_speaking(ms)
        else:
            print("VNR : Sorry, I don't know how to answer that yet ")
            bot_speaking("Sorry, I don't know how to answer that yet")
chat()

VNR :Hi! I am your personal bot. I am here to answer queries on VNRVJIET
YOU : differentiators of vnr
VNR : Sorry, I don't know how to answer that yet 
YOU : differentiators
VNR : Sorry, I don't know how to answer that yet 
YOU : vision
VNR : VNRVJIET VISION-To be a World Class University providing value-based education, conducting interdisciplinary research in cutting edge technologies leading to sustainable socio-economic development of the nation.
YOU : theme
VNR : VJ THEME-Vignana Jyothi as an educational group believes in our significant differentiators.
 Our first differentiator among their top institutions is that while others focus only on one word â€˜successâ€™, we focus on three vital words, empathy, happiness, and success and in this specific order.
 Our faculty are trained accordingly and they have great pride in it.
YOU : facilities in vnr
VNR : 1.The institution has adequate facilities for teaching - learning. viz., classrooms, laboratories, computing equipment, etc 
 2.s

# GUI APPLICATION - CHATBOT

In [ ]:
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"
bot_name ="VNR-BOT"
def get_response(msg):
    #print("Hi! I am your personal bot. I am here to answer queries on VNRVJIET")
    while True:
        inp=msg
        if inp.lower() == "quit"or inp==None:
            break
        inp_x = word_checker(inp)
        results = model.predict([bag_of_words(inp_x, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        #print(results[results_index])
        if results[results_index] >= 0.9:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
                    ms= random.choice(responses)
                    return ms
        else:
            return " Sorry, I don't know how to answer that yet "

class ChatApplication:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        #self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="WELCOME TO VNRVJIET CHAT-BOT", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY, command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
     
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
        
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        bot_speaking(get_response(msg))
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
             
        
if __name__ == "__main__":
    app = ChatApplication()
    app.run()